In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Real_gtzan.keras/src')

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/genres/"

bextract_single.mf  classical  country	hi.mf	  ja.mf  metal	reggae	ro.mf
bl.mf		    cl.mf      di.mf	hiphop	  jazz	 po.mf	re.mf
blues		    co.mf      disco	input.mf  me.mf  pop	rock


In [ ]:
import tensorflow  as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
#Making song data_set-------------------------------------------
import os
import librosa
import itertools
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from scipy.stats import skew


In [ ]:

"""
@description: Method to split a song into multiple songs using overlapping windows
"""
def splitsongs(X, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = 33000
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        if s.shape[0] != chunk:
            continue

        temp_X.append(s)

    return np.array(temp_X)

"""
@description: Method to convert a list of songs to a np array of melspectrograms
"""
def to_melspectrogram(songs, n_fft=1024, hop_length=256):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft=n_fft,
        hop_length=hop_length, n_mels=128)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    # np.array([librosa.power_to_db(s, ref=np.max) for s in list(tsongs)])
    return np.array(list(tsongs))


In [ ]:
def make_dataset_dl(song):
    # Convert to spectrograms and split into small windows
    signal, sr = librosa.load(song, sr=None)

    # Convert to dataset of spectograms/melspectograms
    signals = splitsongs(signal)

    # Convert to "spec" representation
    specs = to_melspectrogram(signals)

    return specs
#Completed Making DataSet------------------------

In [ ]:
#Extracting Majority Voting----------------
import numpy as np


In [ ]:
def majority_voting(scores, dict_genres):
    preds = np.argmax(scores, axis = 1)
    values, counts = np.unique(preds, return_counts=True)
    counts = np.round(counts/np.sum(counts), 2)
    votes = {k:v for k, v in zip(values, counts)}
    votes = {k: v for k, v in sorted(votes.items(), key=lambda item: item[1], reverse=True)}
    return [(get_genres(x, dict_genres), prob) for x, prob in votes.items()]


In [ ]:
def get_genres(key, dict_genres):
    # Transforming data to help on transformation
    labels = []
    tmp_genre = {v:k for k,v in dict_genres.items()}

    return tmp_genre[key]

In [ ]:

from joblib import load
from tensorflow.keras.models import load_model

In [ ]:
genres = {
    'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4, 
    'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9
}

In [ ]:
song="/content/drive/My Drive/Colab Notebooks/Samajavaragamana.mp3"
model="/content/drive/My Drive/Colab Notebooks/Real_gtzan.keras/models/custom_cnn_2d.h5"

In [ ]:
ss=song.split("Colab Notebooks/",1)[1]
print(ss)

Samajavaragamana.mp3


In [ ]:
def dl_run(song,model,genres):
            X = make_dataset_dl(song)
            ss=song.split("Colab Notebooks",1)[1]
            model = load_model(model)
            preds = model.predict(X)
            votes = majority_voting(preds, genres)
            print("{} is a {} song".format(ss[1:], votes[0][0]))
            print("most likely genres are: {}".format(votes[:3]))

In [ ]:
dl_run(song,model,genres)     

Samajavaragamana.mp3 is a blues song
most likely genres are: [('blues', 0.61), ('rock', 0.15), ('country', 0.1)]
